In [ ]:
#crawl link - buying
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 10):
    url = f'https://muaban.net/bat-dong-san/ban-nha-dat-chung-cu#page={i}'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ <a> có class là "title" và thuộc tính href
    links = soup.find_all('a', class_='title', href=True)
    
    # Kiểm tra xem có ít nhất một thẻ a nào không
    if links:
        # Lặp qua các thẻ a và lấy thuộc tính href
        for link in links:
            href = link['href']
            full_url = 'https://muaban.net' + href
            all_urls.append(full_url)
    
# Ghi các URL vào một file văn bản
file_path = "buy_urls.txt"
with open(file_path, "w") as file:
    for url in all_urls:
        file.write(url + "\n")

print("Đã lưu các URL vào file buy_urls.txt")

In [ ]:
#crawl link - renting
from bs4 import BeautifulSoup
import requests

# Khởi tạo một danh sách để lưu trữ các URL
all_urls = []

# Sử dụng vòng lặp để duyệt qua các trang
for i in range(1, 7):
    url = f'https://muaban.net/bat-dong-san/cho-thue-nha-dat?page={i}'
    
    # Gửi yêu cầu GET đến trang web
    response = requests.get(url)
    
    # Parse HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Tìm tất cả các thẻ <a> có class là "title" và thuộc tính href
    links = soup.find_all('a', class_='title', href=True)
    
    # Kiểm tra xem có ít nhất một thẻ a nào không
    if links:
        # Lặp qua các thẻ a và lấy thuộc tính href
        for link in links:
            href = link['href']
            full_url = 'https://muaban.net' + href
            all_urls.append(full_url)
    
# Ghi các URL vào một file văn bản
file_path = "rent_urls.txt"
with open(file_path, "w") as file:
    for url in all_urls:
        file.write(url + "\n")

print("Đã lưu các URL vào file rent_urls.txt")

In [10]:
# crawl data - buying
import json
from bs4 import BeautifulSoup
import requests
import re

def extract_info_from_url(url):
    try:
        # Thực hiện yêu cầu GET để truy cập vào URL
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text

            # Sử dụng BeautifulSoup để phân tích HTML
            soup = BeautifulSoup(html_content, "html.parser")

            # Trích xuất thông tin cần thiết từ HTML
            title = soup.find("h1").text.strip()
            price = soup.find(class_="price").text.strip()
            address = soup.find(class_="address").text.strip()

            # Trích xuất thông tin từ các thẻ <ul> và <li>
            ul_element = soup.find("ul", class_="sc-6orc5o-24 foVitw")

            # Khởi tạo các biến thông tin
            property_type, area, legal_documents, floors, bedrooms, toilets, direction = None, None, None, None, None, None, None

            # Nếu có <ul> chứa thông tin, trích xuất thông tin từ các phần tử <span>
            if ul_element:
                items = ul_element.find_all("li")
                for item in items:
                    label = item.find("span", class_="label")
                    if label:
                        span = item.find("span", string=True, recursive=False)
                        if span:
                            if "Loại hình bất động sản" in label.text:
                                property_type = span.text.strip()
                            elif "Diện tích đất" in label.text:
                                area = span.text.strip()
                            elif "Diện tích sử dụng" in label.text:
                                area = span.text.strip()
                            elif "Giấy tờ pháp lý" in label.text:
                                legal_documents = span.text.strip()
                            elif "Tổng số tầng" in label.text:
                                floors = span.text.strip()
                            elif "Số phòng ngủ" in label.text:
                                bedrooms = span.text.strip()
                            elif "Số phòng vệ sinh" in label.text:
                                toilets = span.text.strip()
                            elif "Hướng cửa chính" in label.text:
                                direction = span.text.strip()

            # Trích xuất thông tin "Chi tiết"
            detail_div = soup.find("div", class_="sc-6orc5o-18 gdAVnx") 

            for child in detail_div.find_all("div", class_="phone-wrapper"):
                child.decompose()  
           
            for tag in detail_div.find_all(True):  # Tìm tất cả các thẻ con
                if tag.name != 'br':  # Nếu tên thẻ không phải là <br>
                    tag.unwrap()  # Giữ lại nội dung của thẻ và loại bỏ thẻ đó

            details = detail_div.decode_contents()
                        
            phone_wrapper = soup.find("div", class_="phone-wrapper")
            if phone_wrapper:
                phone_span = phone_wrapper.find("span", class_="phone-hidden")
                if phone_span and 'data-phone' in phone_span.attrs:
                    phone_number = phone_span['data-phone']
        
            # Trích xuất link ảnh
            image_div = soup.select_one("div.slick-slide.slick-active.slick-current img")
            image_link = None
            if image_div and 'src' in image_div.attrs:
                image_link = image_div['src']
            
            if image_link is None or area is None:
                return None
            
            # Tạo một từ điển chứa thông tin từng bài đăng
            item = {
                "title": title,
                "price": price,                
                "area": area,
                "address": address,
                "description": details,
                "type": "Mua Bán",
                "type_estate": property_type,
                "law": legal_documents,
                "floor": floors,
                "bedroom": bedrooms,
                "toilet": toilets,             
                "direction": direction,   
                "image_links": image_link,
                "phonenumber": phone_number
            }
            return item
        else:
            print(f"Lỗi khi truy cập vào URL: {url}")
            return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

# Tạo một danh sách để lưu trữ thông tin từ mỗi URL
data_list = []

# Đọc nội dung từ tệp muaban.txt
with open("buy_urls.txt", "r", encoding="utf-8") as file:
    urls = file.readlines()

# Lặp qua từng URL để lấy thông tin
for url in urls:
    # Xóa khoảng trắng và ký tự xuống dòng từ URL
    url = url.strip()
    # Trích xuất thông tin từ URL và thêm vào danh sách
    item = extract_info_from_url(url)
    if item:
        data_list.append(item)

# Lưu danh sách dưới dạng JSON vào tệp muaban.json
with open("buy_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp buy_data.json")


Dữ liệu đã được lưu vào tệp buy_data.json


In [ ]:
# crawl data - renting
import json
from bs4 import BeautifulSoup
import requests
import re

def extract_info_from_url(url):
    try:
        # Thực hiện yêu cầu GET để truy cập vào URL
        response = requests.get(url)
        if response.status_code == 200:
            html_content = response.text

            # Sử dụng BeautifulSoup để phân tích HTML
            soup = BeautifulSoup(html_content, "html.parser")
            
                       
            # Trích xuất thông tin cần thiết từ HTML
            title = soup.find("h1").text.strip()
            price = soup.find(class_="price").text.strip()
            address = soup.find(class_="address").text.strip()
            # date_posted = soup.find(class_="date").text.strip()

            # Trích xuất thông tin từ các thẻ <ul> và <li>
            ul_element = soup.find("ul", class_="sc-6orc5o-24 foVitw")

            # Khởi tạo các biến thông tin
            property_type, area, legal_documents, floors, bedrooms, toilets, direction = None, None, None, None, None, None, None

            # Nếu có <ul> chứa thông tin, trích xuất thông tin từ các phần tử <span>
            if ul_element:
                items = ul_element.find_all("li")
                for item in items:
                    label = item.find("span", class_="label")
                    if label:
                        span = item.find("span", string=True, recursive=False)
                        if span:
                            if "Loại hình bất động sản" in label.text:
                                property_type = span.text.strip()
                            elif "Diện tích đất" in label.text:
                                area = span.text.strip()
                            elif "Diện tích sử dụng" in label.text:
                                area = span.text.strip()
                            elif "Giấy tờ pháp lý" in label.text:
                                legal_documents = span.text.strip()
                            elif "Tổng số tầng" in label.text:
                                floors = span.text.strip()
                            elif "Số phòng ngủ" in label.text:
                                bedrooms = span.text.strip()
                            elif "Số phòng vệ sinh" in label.text:
                                toilets = span.text.strip()
                            elif "Hướng cửa chính" in label.text:
                                direction = span.text.strip()

             # Trích xuất thông tin "Chi tiết"
            detail_div = soup.find("div", class_="sc-6orc5o-18 gdAVnx") 

            for child in detail_div.find_all("div", class_="phone-wrapper"):
                child.decompose()
                
            for tag in detail_div.find_all(True):  # Tìm tất cả các thẻ con
                if tag.name != 'br':  # Nếu tên thẻ không phải là <br>
                    tag.unwrap()  # Giữ lại nội dung của thẻ và loại bỏ thẻ đó

            details = detail_div.decode_contents()
            
            phone_wrapper = soup.find("div", class_="phone-wrapper")
            if phone_wrapper:
                phone_span = phone_wrapper.find("span", class_="phone-hidden")
                if phone_span and 'data-phone' in phone_span.attrs:
                    phone_number = phone_span['data-phone']
            
            # Trích xuất link ảnh
            image_div = soup.select_one("div.slick-slide.slick-active.slick-current img")
            image_link = None
            if image_div and 'src' in image_div.attrs:
                image_link = image_div['src']

            # Kiểm tra nếu không có link ảnh thì bỏ qua
            if image_link is None or area is None:
                return None

            # Tạo một từ điển chứa thông tin từng bài đăng
            item = {
                "title": title,
                "price": price,
                "area": area,
                "address": address,
                "description": details,
                "type": "Cho Thuê",
                "type_estate": property_type,
                "law": legal_documents,
                "floor": floors,
                "bedroom": bedrooms,
                "toilet": toilets,
                "direction": direction,
                "image_links": image_link,
                "phonenumber": phone_number
            }                
            return item        
        else:
            print(f"Lỗi khi truy cập vào URL: {url}")
            return None
    except Exception as e:
        print(f"Lỗi khi trích xuất thông tin từ URL: {url}, Lỗi: {e}")
        return None

# Tạo một danh sách để lưu trữ thông tin từ mỗi URL
data_list = []

# Đọc nội dung từ tệp rent_urls.txt
with open("rent_urls.txt", "r", encoding="utf-8") as file:
    urls = file.readlines()

# Lặp qua từng URL để lấy thông tin
for url in urls:
    # Xóa khoảng trắng và ký tự xuống dòng từ URL
    url = url.strip()
    # Trích xuất thông tin từ URL và thêm vào danh sách
    item = extract_info_from_url(url)
    if item:
        data_list.append(item)

# Lưu danh sách dưới dạng JSON vào tệp rent_data.json
with open("rent_data.json", "w", encoding="utf-8") as json_file:
    json.dump(data_list, json_file, ensure_ascii=False, indent=4)

print("Dữ liệu đã được lưu vào tệp rent_data.json")
